In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests
from scipy import stats
import math

In [ ]:
def get_top_systems(drug_name, nest_map, rlipp_df):
    
    subsys_df = rlipp_df.sort_values(by='P_rho', ascending=False, ignore_index=True)
    subsys_df['Rank'] = 0
    subsys_df['Name'] = ''
    subsys_df = subsys_df[['Rank', 'Term', 'Name', 'P_rho', 'P_pval', 'C_rho', 'C_pval', 'RLIPP']]
    for i, row in subsys_df.iterrows():
        subsys_df.at[i, 'Rank'] = i+1
        subsys_df.at[i, 'Name'] = nest_map[row['Term']]
        
    return subsys_df

In [ ]:
def bh(p_vals, alpha):
    res = multipletests(p_vals, alpha=alpha, method='fdr_bh')
    return res[1]

bh.__name__ = 'BH'

In [ ]:
def bonferroni(p_vals):
    res = multipletests(p_vals, alpha=0.05, method='bonferroni')
    return res[1]

bonferroni.__name__ = 'Bonferroni'

In [ ]:
def system_significance(ont, dataset, drug, zscore_method):
    
    rlipp_dict = dict()
    for i in range(1, 101):
        rlipp_file_list = []
        for j in range(1, 6):
            nf = i + 100*(j-1)
            modeldir = '../models/mbb/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(nf)
            rlipp_df = pd.read_csv(modeldir + '/rlipp.out', sep='\t')[['Term', 'P_rho']]
            rlipp_file_list.append(rlipp_df)
        agg_df = pd.concat(rlipp_file_list, ignore_index=True)
        agg_rlipp_df = pd.DataFrame(agg_df.groupby(['Term']).mean()).reset_index()
        for _, row in agg_rlipp_df.iterrows():
            term = row['Term']
            if term not in rlipp_dict:
                rlipp_dict[term] = []
            rlipp_dict[term].append(row['P_rho'])
    
    main_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '.txt', sep='\t')
    main_rlipp_df['t_test'] = 0.0
    main_rlipp_df['Perm_test'] = 0.0
    
    for i, row in main_rlipp_df.iterrows():
        
        true_prho = row['P_rho']
        term = row['Term']
        prho_list = sorted(rlipp_dict[term], reverse=True)
            
        result = stats.ttest_1samp(prho_list, true_prho, alternative='less', nan_policy='raise')
        pval = result.pvalue
        if math.isnan(pval):
            pval = 1.0
        main_rlipp_df.at[i, 't_test'] = pval
        
        for j, prho in enumerate(prho_list):
            if true_prho <= prho:
                continue
            break
        main_rlipp_df.at[i, 'Perm_test'] = j/100
        
    main_rlipp_df['t_test'] = bh(main_rlipp_df['t_test'])
    main_rlipp_df['Perm_test'] = bh(main_rlipp_df['Perm_test'])
        
    return rlipp_dict, main_rlipp_df

In [ ]:
def system_significance_2(ont, dataset, drug, zscore_method, alpha):
    
    mbb_rlipp_dict = dict()
    vnn_rlipp_dict = dict()
    for i in range(1, 6):
        for j in range(1, 101):
            nf = j + 100*(i-1)
            modeldir = '../models/mbb/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(nf)
            rlipp_df = pd.read_csv(modeldir + '/rlipp.out', sep='\t')[['Term', 'P_rho']]
            for _, row in rlipp_df.iterrows():
                term = row['Term']
                if term not in mbb_rlipp_dict:
                    mbb_rlipp_dict[term] = []
                mbb_rlipp_dict[term].append(row['P_rho'])
                
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        rlipp_df = pd.read_csv(modeldir + '/rlipp.out', sep='\t')[['Term', 'P_rho']]
        for _, row in rlipp_df.iterrows():
            term = row['Term']
            if term not in vnn_rlipp_dict:
                vnn_rlipp_dict[term] = []
            vnn_rlipp_dict[term].append(row['P_rho'])
    
    main_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '.txt', sep='\t')
    main_rlipp_df['t_test'] = 0.0
    
    for i, row in main_rlipp_df.iterrows():
        term = row['Term']
        result = stats.ttest_ind(mbb_rlipp_dict[term], vnn_rlipp_dict[term], alternative='less')
        pval = result.pvalue
        if math.isnan(pval):
            pval = 1.0
        main_rlipp_df.at[i, 't_test'] = pval
        
    main_rlipp_df['t_test'] = bh(main_rlipp_df['t_test'], alpha)
        
    return mbb_rlipp_dict, main_rlipp_df

In [ ]:
def get_shuffled_rlipp():
    rlipp_dict = dict()
    for i in range(1, 1001):
        rlipp_file_list = []
        for j in range(1, 6):
            modeldir = '../models/shuffled_input/' + str(j) + '_' + str(i)
            rlipp_df = pd.read_csv(modeldir + '/rlipp.out', sep='\t')[['Term', 'P_rho']]
            rlipp_file_list.append(rlipp_df)
        agg_df = pd.concat(rlipp_file_list, ignore_index=True)
        agg_rlipp_df = pd.DataFrame(agg_df.groupby(['Term']).mean()).reset_index()
        for _, row in agg_rlipp_df.iterrows():
            term = row['Term']
            if term not in rlipp_dict:
                rlipp_dict[term] = []
            rlipp_dict[term].append(row['P_rho'])
            
    return rlipp_dict

In [ ]:
def system_significance_shuffled_input(rlipp_dict, drug, alpha):
    
    main_rlipp_df = pd.read_csv('../models/rlipp/' + drug + '.txt', sep='\t')
    main_rlipp_df['t_test'] = 0.0
    main_rlipp_df['Perm_test'] = 0.0
    for i, row in main_rlipp_df.iterrows():
        
        term = row['Term']
        true_prho = 0.5 #row['P_rho']
        prho_list = sorted(rlipp_dict[term], reverse=True)
        
        result = stats.ttest_1samp(prho_list, true_prho, alternative='less')
        main_rlipp_df.at[i, 't_test'] = result.pvalue
        
        for j, prho in enumerate(prho_list):
            if true_prho <= prho + 1.0/1000:
                continue
            break
        main_rlipp_df.at[i, 'Perm_test'] = j/1000
        
    main_rlipp_df['t_test'] = bh(main_rlipp_df['t_test'], alpha)
    main_rlipp_df['Perm_test'] = bh(main_rlipp_df['Perm_test'], alpha)
        
    return main_rlipp_df

In [ ]:
nest_df = pd.read_csv('../data/NeST/NeST_node.csv', sep=',')
nest_map = {row['name'].replace('.', '-'):row['Annotation'] for i, row in nest_df.iterrows()}

In [ ]:
ont = 'ctg'
dataset = 'av'
zscore_method = 'auc'
folds = 5

drugs = list(pd.read_csv('../data/training_files_av/drugname_av.txt', header=None, names=['D'])['D'])
drugs = ['Palbociclib']
drug = 'Palbociclib'

In [ ]:
for drug in drugs:
    for i in range(1, folds+1):
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        rlipp_df = pd.read_csv(modeldir + '/rlipp.out', sep='\t')
        subsys_df = get_top_systems(drug, nest_map, rlipp_df)
        subsys_df.to_csv(modeldir + '/subsystem_ranks.txt', sep='\t', index=False)

In [ ]:
#Merging p_rho

for drug in drugs:
    agg_terms = []
    for i in range(1, folds+1):
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        subsys_df = pd.read_csv(modeldir + '/subsystem_ranks.txt', sep='\t')[['Term', 'Name', 'P_rho', 'C_rho', 'RLIPP']]
        agg_terms.append(subsys_df)
    
    agg_df = pd.concat(agg_terms, ignore_index=True)
    agg_rlipp_df = pd.DataFrame(agg_df.groupby(['Term', 'Name']).mean()).reset_index()
    agg_rlipp_df = agg_rlipp_df.sort_values(by='P_rho', ascending=False)
    agg_rlipp_df.to_csv('../models/rlipp/' + drug + '.txt', sep='\t', float_format='%.2e', index=False)

In [ ]:
#Merging gene rho with p_values

for drug in ['Palbociclib']:
    agg_terms = []
    for i in range(1, folds+1):
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        subsys_df = pd.read_csv(modeldir + '/gene_rho.out', sep='\t')[['Gene', 'Rho', 'P_val']]
        agg_terms.append(subsys_df)
    
    agg_df = pd.concat(agg_terms, ignore_index=True)
    agg_rlipp_df = pd.DataFrame(agg_df.groupby(['Gene']).agg({'Rho':'mean',
                                                              'P_val': [bh, bonferroni, np.max, np.prod]
                                                             })).reset_index()
    agg_rlipp_df.columns = ['_'.join(col).strip('_') for col in agg_rlipp_df.columns.values]
    agg_rlipp_df = agg_rlipp_df.sort_values(by='Rho_mean', ascending=False)
    agg_rlipp_df.to_csv('../models/rlipp/' + drug + '_gene_rho.txt', sep='\t', float_format='%.4f', index=False)

In [ ]:
#Merging p_rho with p_values

for drug in drugs:
    agg_terms = []
    for i in range(1, folds+1):
        modeldir = '../models/model_' + ont + '_' + dataset + '_' + drug + '_' + zscore_method + '_' + str(i)
        subsys_df = pd.read_csv(modeldir + '/subsystem_ranks.txt', sep='\t')[['Term', 'Name', 'P_rho', 'P_pval']]
        agg_terms.append(subsys_df)
    
    agg_df = pd.concat(agg_terms, ignore_index=True)
    agg_rlipp_df = pd.DataFrame(agg_df.groupby(['Term', 'Name']).agg({'P_rho':'mean', 
                                                                      'P_pval': [bh, bonferroni, np.max, np.prod]
                                                                     })).reset_index()
    agg_rlipp_df.columns = ['_'.join(col).strip('_') for col in agg_rlipp_df.columns.values]
    agg_rlipp_df = agg_rlipp_df.sort_values(by='P_rho_mean', ascending=False)
    agg_rlipp_df.to_csv('../models/rlipp/' + drug + '_all_pval.txt', sep='\t', float_format='%.4f', index=False)

In [ ]:
rlipp_dict, rlipp_df = system_significance_2(ont, dataset, drug, zscore_method, 0.05)

In [ ]:
rlipp_df.to_csv('../models/rlipp/' + drug + '_system_significance_bh_0.3.txt', sep='\t', float_format='%.2e', index=False)

In [ ]:
si_rlipp_dict = get_shuffled_rlipp()

In [ ]:
si_rlipp_df = system_significance_shuffled_input(si_rlipp_dict, drug, 0.05)

In [ ]:
si_rlipp_df.to_csv('../models/rlipp/' + drug + '_system_significance_shuffled_input_0.5.txt', sep='\t', float_format='%.2e', index=False)

In [ ]:
si_rlipp_df.query("Term == 'NEST:50'")